In [ ]:
from expansion_policy_fn import *
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import os
from multiprocessing import Pool
from functools import reduce

if __name__ == '__main__':
    print('Loading data...')
    prod_to_rules = defaultdict(set)
    with open('data/templates_expansion.dat', 'r') as f:
        for l in tqdm(f, desc='products'):
            rule, prod = l.strip().split('\t')
            prod_to_rules[prod].add(rule)



    expansion_rules = {}
    with open('data/expansion_expansion.dat', 'r') as f:
        for i, l in tqdm(enumerate(f), desc='expansion'):
            rule = l.strip()
            expansion_rules[rule] = i

    
    
    save_path = 'saved_models'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Variance threshold
    # No specific threshold is mentioned in the paper,
    # just that it's used to "remove rare features"
    # idx = np.load(os.path.join(save_path, 'expansion.idx.npy'))
    chunk_size = 100
    prods = []
    for prod, rules in prod_to_rules.items():
        if any(r in expansion_rules for r in rules):
            prods.append(prod)
    # prods = random.sample(prods, 200000)    
    with Pool() as p:
        _, _, var = reduce(reducefn, tqdm(p.imap(mapfn, chunker(prods, chunk_size)), total=len(prods)//chunk_size))
    idx = np.where(var > 0)[0]
    print(len(idx))
    np.save(os.path.join(save_path, 'expansion.idx'), idx, allow_pickle=False)
    
    np.save(os.path.join(save_path, 'expansion.var'), var, allow_pickle=False)    

Loading data...


products: 3143138it [00:08, 386495.83it/s]
expansion: 55608it [00:00, 723812.06it/s]
  0%|          | 46/9875 [01:21<2:10:13,  1.26it/s] 

In [13]:
idx1 = np.where(var > 5e-5)[0]
print(len(idx1))

23086


In [3]:
np.save(os.path.join(save_path, 'expansion.var'), var, allow_pickle=False) 

In [4]:
len(var)

1000000

In [14]:
np.save(os.path.join(save_path, 'expansion5e-5.idx'), idx1, allow_pickle=False)